# [[데이콘] 영화 리뷰 감성 분석 PBL2](https://dacon.io/competitions/official/235995/codeshare/6529?page=1&dtype=recent)

In [ ]:
!pip install konlpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import sys
import nltk
from collections import Counter
from wordcloud import WordCloud
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
nltk.download('all')

In [62]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')[['document', 'label']]
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test.csv')[['document']]
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/sample_submission.csv')

In [63]:
train

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [64]:
test

,document
0,굳 ㅋ
1,GDNTOPCLASSINTHECLUB
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네


In [65]:
# train/test 결측치 확인
print('train 결측치 개수\n',train.isnull().sum())
print(train[train['document'].isnull()])
print('\n')
print('test 결측치 개수\n',test.isnull().sum())
print(test[test['document'].isnull()])

train 결측치 개수
 document    5
label       0
dtype: int64
       document  label
25857       NaN      1
55737       NaN      1
110014      NaN      0
126782      NaN      0
140721      NaN      0


test 결측치 개수
 document    3
dtype: int64
      document
5746       NaN
7899       NaN
27097      NaN


In [66]:
# train/test 결측치 대체
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [67]:
# 결측치 개수 확인
print(train.isnull().sum())
print(test.isnull().sum())

document    0
label       0
dtype: int64
document    0
dtype: int64


### 1-3. Feature & Target 설정

In [68]:
train_feature = train.document
train_label = train.label

In [69]:
train_feature.head()

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [70]:
train_feature.tail()

149995                인간이 문제지.. 소는 뭔죄인가..
149996                      평점이 너무 낮아서...
149997    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999           한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, dtype: object

## 2. 텍스트 전처리
### 2-1. CountVectorizer의 이해
- 워드 임베딩 진행
  - 워드 임베딩: 컴퓨터가 계산하기 위해 문장을 숫자로 바꾸는 것(단어를 벡터로 표현)
- 본 분석에서는 CountVectorizer를 이용해 문장을 숫자로 표현
  - CountVectorizer: 입력된 문장을 **토큰화**하여 토큰의 **등장 빈도 벡터**로 바꾸는 기법


In [71]:
# 예시
from sklearn.feature_extraction.text import CountVectorizer

sample_vectorizer = CountVectorizer()
sample_text1 = ["hello, my name is dacon and I am a data scientist!"]
sample_vectorizer.fit(sample_text1) # CountVectorizer 학습

print(sample_vectorizer.vocabulary_) # sample_vectorizer가 학습한 단어 목록

{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'dacon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


길이가 1개인 단어들과 특수문자는 의미가 적다고 판단하여 제외되었음

In [72]:
# 다른 문장을 활용해 벡터로 transform 진행
sample_text2 = ["you are learning dacon data science"]

sample_vector = sample_vectorizer.transform(sample_text2)
print(sample_vector.toarray())

[[0 0 1 1 0 0 0 0 0]]


↑ transform의 결과, 단어들의 출현 빈도로 이루어진 크기 9의 벡터 출력됨

이 벡터를 BoW라고 하는데, BoW란 CountVectorizer로 변환된 단어의 집합을 의미함

sample_text2에는 dacon과 data가 1번씩 출현했기에 해당 인덱스 값이 1로 표현되었고, 등장하지 않은 단어들은 0으로 표시됨

In [73]:
sample_text3 = ["you are learning dacon data science with movie data"]

sample_vector = sample_vectorizer.transform(sample_text3)
print(sample_vector.toarray())

[[0 0 1 2 0 0 0 0 0]]


"dacon" 단어가 1번, "data" 단어가 2번 출현했기에 "dacon" 단어에 해당하는 인덱스에는 1의 값이, "data" 에 해당하는 인덱스에는 2의 값이 할당됨

CountVectorizer 를 하나의 문장을 사용해 학습 시켰기에 생성된 Vocab과 BoW 의 크기는 모두 9였음

큰 데이터를 사용해 CountVectorizer를 학습시키면 Vocab과 BoW의 크기는 증가할 것

In [74]:
vectorizer = CountVectorizer() # countvectorizer 생성
vectorizer.fit(train_feature) # countvectorizer 학습(fit)
train_X = vectorizer.transform(train_feature) # transform

In [75]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_X, train_label)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

위에서 train 데이터로 학습한 모델로 test data를 넣어 예측값을 도출함

fit_transform() 매서드는 학습을 위한 train data에 사용되고 test data에는 transform() 매서드만 사용해야 함

test data에도 fit_transform() 매서드를 사용하는 것은 모델이 test data에 대해서도 학습을 한다는 것을 의미함 → No!!

In [76]:
test_X_vect = vectorizer.transform(test['document']) # test data는 transform() 매서드만 적용

pred = model.predict(test_X_vect)
print(pred)

[0 0 0 ... 1 0 0]


### 2-2. 데이터 정제


In [77]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train)

# 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [78]:
train.head()

,index,document,label
0,149969,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독..,0
1,78,한국영화 흥행코드: 갈등-갈등-계~에속 갈등-화해-감동- 평점 10점 남발- 흥행 ...,0
2,108043,재밌다시간이아깝지않다,1
3,128412,그 우물을 들여다 보면 자신이 비춰진다,1
4,2188,데스노트는 이걸 보고 배워야 한다.,1


In [79]:
val.head()

,document,label
43550,왜케평점이낮냐...진짜재밋게본건데몇번이고..,1
22364,줄 안서서 좋았어 유니버살 워터월드~~~~,1
17324,엑스맨과 비교한 감독발언... 슈퍼맨을 기대했다가 우뢰매를 본 느낌...,0
114584,뷁!,0
9705,마무리가.............내상각그대로네,0


In [80]:
train['preprocessed'] = train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '') # 한글과 공백을 제외하고 모두 제거
train[:5]

<ipython-input-80-f2548bcc9f40>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['preprocessed'] = train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '') # 한글과 공백을 제외하고 모두 제거


,index,document,label,preprocessed
0,149969,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독..,0,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독
1,78,한국영화 흥행코드: 갈등-갈등-계~에속 갈등-화해-감동- 평점 10점 남발- 흥행 ...,0,한국영화 흥행코드 갈등갈등계에속 갈등화해감동 평점 점 남발 흥행 뻔하지 뭐
2,108043,재밌다시간이아깝지않다,1,재밌다시간이아깝지않다
3,128412,그 우물을 들여다 보면 자신이 비춰진다,1,그 우물을 들여다 보면 자신이 비춰진다
4,2188,데스노트는 이걸 보고 배워야 한다.,1,데스노트는 이걸 보고 배워야 한다


In [81]:
train['preprocessed'] = train['preprocessed'].str.replace(' +', ' ') # 다중 공백 제거
train[:5]

<ipython-input-81-382d39dcefb0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['preprocessed'] = train['preprocessed'].str.replace(' +', ' ') # 다중 공백 제거


,index,document,label,preprocessed
0,149969,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독..,0,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독
1,78,한국영화 흥행코드: 갈등-갈등-계~에속 갈등-화해-감동- 평점 10점 남발- 흥행 ...,0,한국영화 흥행코드 갈등갈등계에속 갈등화해감동 평점 점 남발 흥행 뻔하지 뭐
2,108043,재밌다시간이아깝지않다,1,재밌다시간이아깝지않다
3,128412,그 우물을 들여다 보면 자신이 비춰진다,1,그 우물을 들여다 보면 자신이 비춰진다
4,2188,데스노트는 이걸 보고 배워야 한다.,1,데스노트는 이걸 보고 배워야 한다


### 2-3. 토큰화
- 토큰화(Tokenization): 말뭉치(Corpus)를 주어진 단위(Token)로 나누는 과정
  1. 문장 토큰화
  1. 단어 토큰화

In [82]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." 

from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>',sent_tokenize(text))

from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']
단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


In [83]:
from konlpy.tag import Okt 

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


In [84]:
tokenized = [] # 데이터프레임의 한 칼럼의 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내어기
  tokens = okt.morphs(sentence, stem=True) # 형태소 분석(stem=True로 어간 추출 진행)
  tokenize = " ".join(tokens) # tokens라는 리스트 안에 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
  tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가
train['tokenized_stem'] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 칼럼명으로 추가

train.head() # 데이터 확인

,index,document,label,preprocessed,tokenized_stem
0,149969,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독..,0,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독,데스페라도 는 시원하다 보다 역시 후속작 은 못 만들다 감독
1,78,한국영화 흥행코드: 갈등-갈등-계~에속 갈등-화해-감동- 평점 10점 남발- 흥행 ...,0,한국영화 흥행코드 갈등갈등계에속 갈등화해감동 평점 점 남발 흥행 뻔하지 뭐,한국영 화 흥행 코드 갈등 갈등 계 에 속 갈등 화해 감동 평점 점 남발 흥행 뻔하다 뭐
2,108043,재밌다시간이아깝지않다,1,재밌다시간이아깝지않다,재밌다 시간 이 아깝다 않다
3,128412,그 우물을 들여다 보면 자신이 비춰진다,1,그 우물을 들여다 보면 자신이 비춰진다,그 우물 을 들이다 보다 자신 이 비추다
4,2188,데스노트는 이걸 보고 배워야 한다.,1,데스노트는 이걸 보고 배워야 한다,데스노트 는 이 걸 보고 배우다 하다


### 2-4. 품사 태깅(POP Tagging)
- 품사 태깅: 주어진 텍스트를 형태소 단위로 나눈 뒤, 각 형태소에 해당 품사를 태깅하여 리스트화 하는 것

In [85]:
# 품사 태깅
print(okt.pos("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [86]:
print(okt.pos("이것은 1점이 아니다 11점을 주고싶은 내 간절한 마음이다"))

[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('1', 'Number'), ('점', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('11', 'Number'), ('점', 'Noun'), ('을', 'Josa'), ('주고싶은', 'Verb'), ('내', 'Noun'), ('간절한', 'Adjective'), ('마음', 'Noun'), ('이다', 'Josa')]


In [87]:
# 명사 추출
print(okt.nouns("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

['오늘', '코딩', '자연어', '처리', '고수']


In [88]:
main_pos = []
for sentence in train['document']:
  pos = okt.pos(sentence)
  main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
  main_words_str = " ".join(main_words)
  main_pos.append(main_words_str)
train["main_pos"] = pd.DataFrame(main_pos)

train.head() 

,index,document,label,preprocessed,tokenized_stem,main_pos
0,149969,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독..,0,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독,데스페라도 는 시원하다 보다 역시 후속작 은 못 만들다 감독,데스페라도 시원하게 봤다만 역시 후속작 만드는 감독
1,78,한국영화 흥행코드: 갈등-갈등-계~에속 갈등-화해-감동- 평점 10점 남발- 흥행 ...,0,한국영화 흥행코드 갈등갈등계에속 갈등화해감동 평점 점 남발 흥행 뻔하지 뭐,한국영 화 흥행 코드 갈등 갈등 계 에 속 갈등 화해 감동 평점 점 남발 흥행 뻔하다 뭐,한국영 흥행 코드 갈등 갈등 계 속 갈등 화해 감동 평점 점 남발 흥행 뻔하지 뭐
2,108043,재밌다시간이아깝지않다,1,재밌다시간이아깝지않다,재밌다 시간 이 아깝다 않다,재밌다 시간 아깝지 않다
3,128412,그 우물을 들여다 보면 자신이 비춰진다,1,그 우물을 들여다 보면 자신이 비춰진다,그 우물 을 들이다 보다 자신 이 비추다,그 우물 들여다 보면 자신 비춰진다
4,2188,데스노트는 이걸 보고 배워야 한다.,1,데스노트는 이걸 보고 배워야 한다,데스노트 는 이 걸 보고 배우다 하다,데스노트 걸 보고 배워야 한다


마지막에 생성된 main_pos 칼럼을 이용해 모델 학습

## 3. 모델링

In [89]:
X_train = train.main_pos
y_train = train.label

### 3.1 검증 데이터셋 전처리
먼저 검증 셋 val에 train 셋과 동일한 전처리 과정을 거쳐줍니다.

In [95]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

val.head()

<ipython-input-95-ee5472298456>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
<ipython-input-95-ee5472298456>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거


,document,label,preprocessed,tokenized_stem,main_pos
43550,왜케평점이낮냐...진짜재밋게본건데몇번이고..,1,왜케평점이낮냐진짜재밋게본건데몇번이고,NaN,NaN
22364,줄 안서서 좋았어 유니버살 워터월드~~~~,1,줄 안서서 좋았어 유니버살 워터월드,왜 난 아직 흑백 이지,왜 난 아직 흑백
17324,엑스맨과 비교한 감독발언... 슈퍼맨을 기대했다가 우뢰매를 본 느낌...,0,엑스맨과 비교한 감독발언 슈퍼맨을 기대했다가 우뢰매를 본 느낌,시작 하다 때 동성애 부분 이 조금 불편하다 정말 편견 을 버리다 해주다 좋다 작품 밉다,시작 할 때 동성애 부분 조금 불편했지만 정말 편견 버리게 해주는 좋은 작품 미다
114584,뷁!,0,뷁,NaN,NaN
9705,마무리가.............내상각그대로네,0,마무리가내상각그대로네,이 영화 를 보기 전 으로 돌아가다 나 는 아무 생각 이 없다 때문 이다,이 영화 보기 전 돌아가싶다 나 아무 생각 없기 때문


In [96]:
# tokenized_stem
tokenized = []

for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

val.head()

,document,label,preprocessed,tokenized_stem,main_pos
43550,왜케평점이낮냐...진짜재밋게본건데몇번이고..,1,왜케평점이낮냐진짜재밋게본건데몇번이고,NaN,NaN
22364,줄 안서서 좋았어 유니버살 워터월드~~~~,1,줄 안서서 좋았어 유니버살 워터월드,왜 난 아직 흑백 이지,왜 난 아직 흑백
17324,엑스맨과 비교한 감독발언... 슈퍼맨을 기대했다가 우뢰매를 본 느낌...,0,엑스맨과 비교한 감독발언 슈퍼맨을 기대했다가 우뢰매를 본 느낌,시작 하다 때 동성애 부분 이 조금 불편하다 정말 편견 을 버리다 해주다 좋다 작품 밉다,시작 할 때 동성애 부분 조금 불편했지만 정말 편견 버리게 해주는 좋은 작품 미다
114584,뷁!,0,뷁,NaN,NaN
9705,마무리가.............내상각그대로네,0,마무리가내상각그대로네,이 영화 를 보기 전 으로 돌아가다 나 는 아무 생각 이 없다 때문 이다,이 영화 보기 전 돌아가싶다 나 아무 생각 없기 때문


In [97]:
# main_pos
main_pos = []

for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,document,label,preprocessed,tokenized_stem,main_pos
43550,왜케평점이낮냐...진짜재밋게본건데몇번이고..,1,왜케평점이낮냐진짜재밋게본건데몇번이고,NaN,NaN
22364,줄 안서서 좋았어 유니버살 워터월드~~~~,1,줄 안서서 좋았어 유니버살 워터월드,왜 난 아직 흑백 이지,왜 난 아직 흑백
17324,엑스맨과 비교한 감독발언... 슈퍼맨을 기대했다가 우뢰매를 본 느낌...,0,엑스맨과 비교한 감독발언 슈퍼맨을 기대했다가 우뢰매를 본 느낌,시작 하다 때 동성애 부분 이 조금 불편하다 정말 편견 을 버리다 해주다 좋다 작품 밉다,시작 할 때 동성애 부분 조금 불편했지만 정말 편견 버리게 해주는 좋은 작품 미다
114584,뷁!,0,뷁,NaN,NaN
9705,마무리가.............내상각그대로네,0,마무리가내상각그대로네,이 영화 를 보기 전 으로 돌아가다 나 는 아무 생각 이 없다 때문 이다,이 영화 보기 전 돌아가싶다 나 아무 생각 없기 때문


### 3-2. LogisticRegression 모델 정의

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() # countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

# 3-3. 모델 학습

In [92]:
from sklearn.linear_model import LogisticRegression #모델 불러오기

model = LogisticRegression()
model.fit(X_train_vec, y_train) # 모델 학습

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [93]:
# 학습한 모델로 에측할 X를 벡터화 진행
X_val = val.main_pos
y_val = val.label

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

ValueError: ignored

### 3-4. 모델 예측

In [ ]:
y_pred = model.predict(X_val_vec)
print(y_pred)

# 0:negative 1:positive

### 3-5. 검증

In [ ]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred)) # 정확도 확인

## 4. 제출